In [1]:
import sys,os
import datetime 
import warnings
import subprocess
import numpy        as np
import pandas       as pd
import xarray       as xr
print('Library have loaded!')

Library have loaded!


In [3]:
## Information of path and loop index
path_cfs_monthly  = '/Volumes/ssd3/DATA/CFS/regrid_full_monthly/'
path_climatology  = '/Volumes/ssd3/DATA/CFS/regrid_full_monthly/climatology_2012-2023/'
path_static       = '/Volumes/ssd3/DATA/static/'
MASK_US           = xr.open_dataset(f'{path_static}US_MASK_logic.nc')['MASK']
var_names         = ['T2MAX','T2MIN','PRAVG','Z500','ASWDNS']
years             = range(2012,2023+1)
months            = range(1,12+1)

# days that CWRF hindcast runs for each month. It is the right one.
days_per_month    = {
    1: [ 1, 6,11,16,21,26,31],
    2: [ 5,10,15,20,25],
    3: [ 2, 7,12,17,22,27],
    4: [ 1, 6,11,16,21,26],
    5: [ 1, 6,11,16,21,26,31],
    6: [ 5,10,15,20,25,30],
    7: [ 5,10,15,20,25,30],
    8: [ 4, 9,14,19,24,29],
    9: [ 3, 8,13,18,23,28],
    10:[ 3, 8,13,18,23,28],
    11:[ 2, 7,12,17,22,27],
    12:[ 2, 7,12,17,22,27],
}

month_to_delete   = {
    1:[8,9,10,11,12],
    2:[9,10,11,12, 1],
    3:[10,11,12, 1, 2],
    4:[11,12,1, 2, 3],
    5:[12,1,2, 3, 4],
    6:[1,2,3, 4, 5],
    7:[2,3,4, 5, 6],
    8:[3,4,5, 6, 7],
    9:[4,5,6, 7, 8],
    10:[5,6,7,8, 9],
    11:[6,7,8,9,10],
    12:[7,8,9,10,11],
}


In [14]:
## Calculate the climatology of CFS
for var_name  in var_names:
    for month in months:
        days     = days_per_month[month]
        for day  in days:
            list_files = []
            for year in years:
                file_name = f'{path_cfs_monthly}{var_name}/CFS_full_time_{var_name}_{year}-{month:02}-{day:02}-06.nc'
                if os.path.exists(file_name):
                    list_files.append(file_name)
            cfs_ds = xr.open_mfdataset(list_files)
            # Delete redundent coords
            if 'heightAboveGround' in cfs_ds.coords:
                cfs_ds      = cfs_ds.reset_coords('heightAboveGround',drop=True)
            # Calculate the climatology
            cfs_climatology = cfs_ds.groupby('time.month').mean(dim='time')
            # Drop month which NOAA not released for downscaling
            filtered_climatology = cfs_climatology.sel(month=~cfs_climatology.month.isin(month_to_delete[month]))
            # Save the data to local disk
            filtered_climatology.to_netcdf(f'{path_climatology}/CFS_climatology_{var_name}_{years[0]}-{years[-1]}_{month:02}{day:02}-06.nc')

OSError: no files to open